<a href="https://colab.research.google.com/github/alcidescoutinho/pytorch_knn-impute_automobile/blob/main/automobile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introdução dos dados

This data set consists of three types of entities:
 
(a) the specification of an auto in terms of various characteristics 

(b) its assigned insurance risk rating 

(c) its normalized losses in use as compared to other cars.  

The second rating corresponds to the
degree to which the auto is more risky than its price indicates.
Cars are initially assigned a risk factor symbol associated with its
price.   Then, if it is more risky (or less), this symbol is
adjusted by moving it up (or down) the scale.  Actuarians call this
process "symboling".  A value of +3 indicates that the auto is
risky, -3 that it is probably pretty safe.


The third factor is the relative average loss payment per insured
  vehicle year.  This value is normalized for all autos within a
  particular size classification (two-door small, station wagons,
  sports/speciality, etc...), and represents the average loss per car
  per year.



Localização do arquivo: https://archive.ics.uci.edu/ml/datasets/Automobile

Com base na explicação da introdução, esse modelo busca alcançar a seguinte lista de objetivos:
1. Preencher os vazios utilizando o KNN impute
2. Criar uma rede neural utilizando pytorch para a previsão da regressão.
3. Fazer um comparativo dos dados previstos com os reais.

# Importação

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
strings = ['symboling', 'normalized-losses', 'make',
           'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels',
           'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 
           'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
           'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']
strings_mod = []

for i in strings:
  ii = i.replace('-','_')
  strings_mod.append(ii)
strings = strings_mod

dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Projetos Portfólio/Regressão/imports-85 (1).csv',
                      names=strings,
                      header=None, 
                      na_values='?')

# Processamento de dados

Processamento de dados 

1. Visualização dos Unicos das features de classificação.
2. Eliminar dados descrepantes
3. Preencher Vázios
4. Obter dados estatísticos e visualização de correlações

Previsão de dados

- obs: - Como o que irei utilizar de previsão é a coluna *normalized-losses*, irei separar os dados vázios para fazer a previsão, enquanto testo o previsor pelo cross-val com os outros modelos. 

1. Padronização das features
2. Seleção das melhores
3. Aplicação do ML

# Funções


- Função que possíbilita verificar diversas características dos dados

In [ ]:
def tabela_resumo(df):
  import plotly.express as px
  figura = df.isnull().sum().sort_values(ascending=True)

  print(f'\n\n               Dataset Shape: {df.shape}\n\n')
  print(f'               Dataset Duplicated: {df.duplicated().sum()}\n\n')
  summary = pd.DataFrame(df.dtypes,columns=['dtypes'])
  summary = summary.reset_index()
  summary['Name'] = summary['index']
  summary = summary[['Name','dtypes']]
  summary['Missing'] = df.isnull().sum().values
  summary['Unique'] = df.nunique().values
  l = []
  r = []
  for i in summary['Name']:
      l.append(df[i].value_counts().index[0])
      r.append(df[i].value_counts().values[0])
  max, min, mean, std, median = [], [], [], [], []
  for n,i in enumerate(summary['dtypes']):
    if (i == 'float64') | (i == 'int64'):
      min.append(round(df.iloc[:,n].min(),3))
      max.append(round(df.iloc[:,n].max(),3))
      mean.append(round(df.iloc[:,n].mean(),3))
      std.append(round(df.iloc[:,n].std(),3))
      median.append(round(df.iloc[:,n].median(),3))
    else:
      min.append(f'class : {i}')
      max.append(f'class : {i}')
      mean.append(f'class : {i}')
      std.append(f'class : {i}')
      median.append(f'class : {i}')
  summary['max'] = max
  summary['min'] = min
  summary['mean'] = mean
  summary['std'] = std
  summary['median'] = median

  summary['Most repeated'] = l
  summary['Repeated counts'] = r
  summary['First Value'] = df.loc[0].values
  summary['Second Value'] = df.loc[1].values

  if df.isnull().sum().sum() > 0:
    fig = px.bar(figura[figura>0], 
                orientation='h', 
                text_auto='.0f', 
                color=figura[figura>0].index,
                color_discrete_sequence = px.colors.qualitative.Set2)
    fig.update_traces(textfont_size=12, textangle=0, textposition="outside", cliponaxis=False)

    fig.show()
  return summary

In [ ]:
"""
Se a quantidade de unicos for maior do que 75%, considero esse dados como float,
caso contrario, será considerado um dado classificável.

"""
def unicos(df):
  qual = list()
  for i in df:
    if (len(df[i].unique()) / len(df)) < 0.75:
      print(f'\nModelo: {i} \n   -> {df[i].unique()}')
    else:
      qual.append(i)
  print(f'Dados quantitativos : {qual}')
  return qual

# KNN Impute
Com a análise de dados, possível verificar que todas as linhas estão ok e 7 features tem dados faltantes.
- Mudar formato da coluna número de portas.
- Usar o KNNImputer para preencher as features vazias.
- Atualizar o DataSet

In [ ]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [ ]:
local = dataset.columns.get_loc('num_of_doors')
dados_loc = list(range(18,dataset.shape[1]))
dados_loc.insert(0, local)
teste_knn = dataset.copy()
modelo = teste_knn.iloc[:,dados_loc].copy()
modelo['num_of_doors'] = modelo['num_of_doors'].replace({'two': 2,'four': 4}).tolist()

# Aplicação no KNN
impute_knn = KNNImputer(n_neighbors=6)
novo_knn = pd.DataFrame(np.round(impute_knn.fit_transform(modelo), 2), columns=modelo.columns)

# Adequar corretamente esses modelos para os casos de dados qualitativoe e quantitativos.

  # Separação dos indices faltantes
faltantes = list()
for i in modelo:
  a = modelo.loc[modelo[i].isnull()].index.tolist()
  if len(a) > 0:
    faltantes.append([i,a])

"""
Para os tipos de classificação, foi substituido pelo mais próximo.
Para os tipos numericos, vai ser substituido pelo valor previsto.
"""
acumulor = list()
for n,i in enumerate(faltantes):
  coluna = i[0]
  numero = i[1]

  if coluna != 'price':
    a = novo_knn[coluna][numero].tolist()
    lista_temporaria = np.round(np.sort(modelo[coluna].unique()),2)
    adicionar = list()

    for ii in a:
      adicionar.append(min(lista_temporaria, key=lambda x:abs(x-ii)))

    posicao = modelo.columns.get_loc(coluna)
    acumulor.append(adicionar)
    print(adicionar)

    for nn, iii in enumerate(numero):
      modelo.iloc[iii, posicao] = adicionar[nn]

  else:
    a = novo_knn[coluna][numero].tolist()
    print(a)
    posicao = modelo.columns.get_loc(coluna)

    for n,ii in enumerate(numero):
      modelo.iloc[ii,posicao] = a[n]

[2.0, 2.0]
[3.24, 3.24, 3.39, 3.35]
[3.29, 3.23, 3.27, 3.21]
[143.0, 134.0]
[5750.0, 5500.0]
[16975.5, 10760.0, 10760.0, 23200.67]


Passamos novamente na função tabela_resumo para verificar se está com dados faltantes e a formatação correta.

In [ ]:
dataset.isnull().sum().sum()

59

In [ ]:
tabela_resumo(modelo)



               Dataset Shape: (205, 9)


               Dataset Duplicated: 5




,Name,dtypes,Missing,Unique,max,min,mean,std,median,Most repeated,Repeated counts,First Value,Second Value
0,num_of_doors,float64,0,2,4.00,2.00,3.112,0.996,4.00,4.00,114,2.00,2.00
1,bore,float64,0,38,3.94,2.54,3.329,0.271,3.31,3.62,23,3.47,3.47
2,stroke,float64,0,36,4.17,2.07,3.255,0.314,3.29,3.40,20,2.68,2.68
3,compression_ratio,float64,0,32,23.00,7.00,10.143,3.972,9.00,9.00,46,9.00,9.00
4,horsepower,float64,0,59,288.00,48.00,104.590,39.665,95.00,68.00,19,111.00,111.00
5,peak_rpm,float64,0,23,6600.00,4150.00,5130.244,479.673,5200.00,5500.00,38,5000.00,5000.00
6,city_mpg,int64,0,29,49.00,13.00,25.220,6.542,24.00,31.00,28,21.00,21.00
7,highway_mpg,int64,0,30,54.00,16.00,30.751,6.886,30.00,25.00,19,27.00,27.00
8,price,float64,0,189,45400.00,5118.00,13250.386,7907.814,10595.00,8921.00,2,13495.00,16500.00


realizar a substituição no DataSet e novamente passar na tabela resumo para verificar se está tudo de acordo com o planejado.

In [ ]:
for i in modelo:
  dataset[i] = modelo[i].tolist()

In [ ]:
tabela_resumo(dataset)

# Machine Learning

1. Realizar a separação do dado a ser previsto e dividir os vazios como a previsão.
2. Utilizar o PyTorch para a criação de uma rede neural para previsão da feature normalized losses.




**Separação dos dados**

In [ ]:
y = dataset['normalized_losses']
x = dataset.drop('normalized_losses',axis=1)

Tratando os dados para passar ao regressor.
1. Por na formatação obj os dados qualitativos e float os qualitativos.
2. Label encoder para os dados qualitativos

In [ ]:
qual = unicos(x)


Modelo: symboling 
   -> [ 3  1  2  0 -1 -2]

Modelo: make 
   -> ['alfa-romero' 'audi' 'bmw' 'chevrolet' 'dodge' 'honda' 'isuzu' 'jaguar'
 'mazda' 'mercedes-benz' 'mercury' 'mitsubishi' 'nissan' 'peugot'
 'plymouth' 'porsche' 'renault' 'saab' 'subaru' 'toyota' 'volkswagen'
 'volvo']

Modelo: fuel_type 
   -> ['gas' 'diesel']

Modelo: aspiration 
   -> ['std' 'turbo']

Modelo: num_of_doors 
   -> [2. 4.]

Modelo: body_style 
   -> ['convertible' 'hatchback' 'sedan' 'wagon' 'hardtop']

Modelo: drive_wheels 
   -> ['rwd' 'fwd' '4wd']

Modelo: engine_location 
   -> ['front' 'rear']

Modelo: wheel_base 
   -> [ 88.6  94.5  99.8  99.4 105.8  99.5 101.2 103.5 110.   88.4  93.7 103.3
  95.9  86.6  96.5  94.3  96.  113.  102.   93.1  95.3  98.8 104.9 106.7
 115.6  96.6 120.9 112.  102.7  93.   96.3  95.1  97.2 100.4  91.3  99.2
 107.9 114.2 108.   89.5  98.4  96.1  99.1  93.3  97.   96.9  95.7 102.4
 102.9 104.5  97.3 104.3 109.1]

Modelo: length 
   -> [168.8 171.2 176.6 177.3 192.7 178.2 1

In [ ]:
def transformador(x, qual):
  # Importações
  from sklearn.preprocessing import LabelEncoder, StandardScaler

  # Selecionando as variáveis qualitativas
  indices_qual = set(range(len(x.columns)))

  for i in qual:
    atual = set([int(x.columns.get_loc(i))])
    indices_qual = indices_qual - atual
  indices_qualitativos = list(indices_qual)

  # Label Encoder
  novo_x = x.copy()
  lb = LabelEncoder()
  for i in x.iloc[:,indices_qualitativos]:
    novo_x[i] = lb.fit_transform(x[i])
  
  # Padronização
  std = StandardScaler()
  std_x = std.fit_transform(novo_x)
  for n,i in enumerate(novo_x):
    novo_x[i] = std_x[:,n]
  return novo_x

In [ ]:
x = transformador(x, qual)

In [ ]:
indice_completo = y.loc[(y.notnull())].index.tolist()
ytr = y.iloc[indice_completo]
xtr = x.iloc[indice_completo,:]
indice = set(y.index.tolist())
indice_previsor = list(indice - set(indice_completo))
x_previsor = x.iloc[indice_previsor, :]


In [ ]:
x_previsor

**Função do pytorch**

In [ ]:
def regressor_pytorch(xtr, xte, ytr, epocas, batch):
  """ 
  epocas = epocas a ser treinado
  batch = tamanho da separação do mesmo dado para um mini treinamento

  Para essa função, a seguinte ordem será seguida:

  1. Transformar os dados de entrada para o formato tensor
  2. Criar a arquitetura da rede neural
  3. Criar o Dataset E batch_loader
  4. Criar a função erro e escolher o gradiente 
  5. Fazer os loopings 
  """
  # Importações
  import torch 
  from torch import nn, optim
  import torch.utils.data as data

  # Transformaçao
  xtr = torch.tensor(np.array(xtr), dtype=torch.float)
  ytr = torch.tensor(np.array(ytr), dtype=torch.float).view(-1,1) # View transforma de [] para [[]]
  xte = torch.tensor(np.array(xte), dtype=torch.float)

  # Arquitetura
  regressor = nn.Sequential(
          nn.Linear(25, 16),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(16, 16),
          nn.ReLU(),
          nn.Dropout(0.2),
          nn.Linear(16,1)
      )


  #Dataset
  dataset = data.TensorDataset(xtr, ytr)

  batch_loader = data.DataLoader(dataset,
                                 batch_size=batch,
                                 shuffle=True)
  
  # Erro e Gradiente
  criterion = nn.L1Loss()

  optimizer = optim.SGD(regressor.parameters(),
                        lr=0.001)
  
  # LOOPING

  for epochs in range(epocas):
    running_loss = 0.

    for batch_set in batch_loader:
      inputs, labels = batch_set

      optimizer.zero_grad()

      outputs = regressor.forward(inputs)
      loss = criterion(outputs, labels)

      loss.backward()
      optimizer.step()

      running_loss += loss.item()

    error = running_loss / len(batch_loader)
    if epochs % 100 == 0:
      print(f'---- Epoca : {epochs} ------- Erro : {error}')
    
  regressor.eval()
  saida = regressor(xte)
  print(f'\n\n\nPara os dados de treino --- Média : {ytr.mean()} --- STD : {ytr.std():.1f}')
  print(f'Para os dados previstos --- Média : {saida.mean():.1f} --- STD : {saida.std():.1f}')
  return saida

# Conclusão 
- Como os 2 primeiros objetivos alcançados, rodamos os dados na rede neural.
- Como foi feita uma separação entre os dados faltantes e não temos como saber o erro, devido a falta de valores verdadeiros, será utilizado a média dos resultados e o desvio padrão, tanto da média dos dados reais como os dos previsoes, além também de levar em consideração o erro médio absoluto atingido ao final a execução
- Os dados, por essas duas medidas, se mostraram muito próximos a realidade, além de apresentar um erro 

In [ ]:
previsao = regressor_pytorch(xtr, x_previsor, ytr, 300, 20)

---- Epoca : 0 ------- Erro : 122.38795725504558
---- Epoca : 100 ------- Erro : 117.64518313937717
---- Epoca : 200 ------- Erro : 45.71936374240451



Para os dados de treino --- Média : 122.0 --- STD : 35.4
Para os dados previstos --- Média : 127.7 --- STD : 27.4
